In [110]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import copy

from general_ode_solver import FitzHughNagumoModel, NumericalSolver

#TODO How to test here: from data_generation.simulations import simulator as s

## Test Usage FitzHughNagumo Model with Solver

In [111]:
fhnM = FitzHughNagumoModel(control_params=['I'])
fhnM.get_config()

{'model': 'FitzHughNagumoModel',
 'x_dim': 2,
 'control_dim': 1,
 'control_params': ['I'],
 'a': 0.7,
 'b': 1.6,
 'epsilon': 0.08,
 'I': None}

In [112]:
control_paramsdict = {'I': np.array([0.35, 0.6])} 
control = np.array([[[0.35], [0.6]],[[0.35], [0.6]]]) #shape(num_steps, n_samples, control_dim)

t = 0
Z = np.array([1,0, 0, 1])
X = np.array([[1,0],[0,1]])
fhnM.odes_vectorized(t, Z, control_paramsdict)

array([ 1.01666667, -0.4       ,  0.136     , -0.072     ])

In [113]:
solver = NumericalSolver(fhnM)
solver.get_derivative(X, control_paramsdict)

array([[ 1.01666667,  0.136     ],
       [-0.4       , -0.072     ]])

In [114]:
num_steps = 2
e =solver.step(X, control, 0.5, num_steps)
e

array([[[ 1.        ,  0.        ],
        [ 0.        ,  1.        ]],

       [[ 1.44907584,  0.07517142],
        [-0.24755332,  0.96060908]],

       [[ 1.68892076,  0.15919073],
        [-0.61105003,  0.91186921]]])

## First Experiments FHN Model

In [115]:
def fhn(t, z, a, b, epsilon, I):
    v, w = z
    dvdt = v - (v**3) / 3 - w + I
    dwdt = epsilon * (v + a - b * w)
    return [dvdt, dwdt]

# Parameters
a = 0.7
b = 1.6
epsilon = 0.08

# Bifurcation parameter values for different regimes
I_values = [0.2, 0.35, 1.2]  # Monostable, Bistable, Oscillatory

# Time span for integration
t_span = (0, 1000)
t_eval = np.linspace(t_span[0], t_span[1], 1001)

# Initial conditions for phase space trajectories
in_con = [
    [-1.5, -1], [0.5, 0], [1.5, 1]  # Different starting points in phase space
]



In [116]:
random_array = np.random.rand(1000, 2)

In [117]:
sol_for_loop = []
for initcon in random_array:
    sol = solve_ivp(fhn, t_span, initcon, args=(a, b, epsilon, I_values[0]), t_eval=t_eval)
    sol_for_loop.append(sol)
       

In [118]:
def ode_vectorized(t, Z, a, b, epsilon, I):
    v, w = Z[:int(len(Z)/2)], Z[int(len(Z)/2):]  
    I_v = np.full_like(v, I)
    a_v = I = np.full_like(w, a)
    dvdt = v - (v**3) / 3 - w + I_v
    dwdt = epsilon * (v + a - b * w)
    return np.append(dvdt, dwdt)  # Return as a 1D array



In [119]:
initcon = random_array.transpose().flatten()
sol_vect = solve_ivp(ode_vectorized, t_span, initcon, args=(a, b, epsilon, I_values[0]), t_eval=t_eval)

In [120]:
sol_vect2 = np.zeros((1000+1,) + initcon.shape)
sol_vect2[0] = initcon
initcon1 = copy.deepcopy(initcon)


for i in range(0,1000):
    
    t_span1 = (i, i+1)
            
    solv = solve_ivp(ode_vectorized, t_span1, initcon1, args=(a, b, epsilon, I_values[0]), t_eval= t_span1)
            
    #Update Initial Condition
    initcon1 = solv.y[:,-1]
    print(solv.y)
    sol_vect2[i+1] = solv.y[:,-1]

[[ 0.19260168 -0.37405229]
 [ 0.9484801   1.27260067]
 [ 0.04584933 -0.48416178]
 ...
 [ 0.39897278  0.42633527]
 [ 0.1902498   0.30220204]
 [ 0.08980027  0.21635948]]
[[-0.37405229 -1.39846093]
 [ 1.27260067  1.39841637]
 [-0.48416178 -1.38217457]
 ...
 [ 0.42633527  0.45866254]
 [ 0.30220204  0.43100152]
 [ 0.21635948  0.36028456]]
[[-1.39846093 -1.81214371]
 [ 1.39841637  1.39494194]
 [-1.38217457 -1.7529619 ]
 ...
 [ 0.45866254  0.50187006]
 [ 0.43100152  0.54972673]
 [ 0.36028456  0.4912392 ]]
[[-1.81214371 -1.80690091]
 [ 1.39494194  1.34518848]
 [-1.7529619  -1.75622627]
 ...
 [ 0.50187006  0.5620135 ]
 [ 0.54972673  0.65084826]
 [ 0.4912392   0.60250467]]
[[-1.80690091 -1.7558857 ]
 [ 1.34518848  1.28028775]
 [-1.75622627 -1.71024351]
 ...
 [ 0.5620135   0.63634115]
 [ 0.65084826  0.73472466]
 [ 0.60250467  0.69507966]]
[[-1.7558857  -1.70402616]
 [ 1.28028775  1.20909208]
 [-1.71024351 -1.66198557]
 ...
 [ 0.63634115  0.71247714]
 [ 0.73472466  0.80309455]
 [ 0.69507966  0.771

In [121]:
sol_vect.y

array([[ 0.19260168, -0.37390441, -1.39827538, ..., -1.29439228,
        -1.29439228, -1.29439228],
       [ 0.9484801 ,  1.27258831,  1.39843871, ..., -1.29439228,
        -1.29439228, -1.29439228],
       [ 0.04584933, -0.4840925 , -1.38211047, ..., -1.29439228,
        -1.29439228, -1.29439228],
       ...,
       [ 0.39897278,  0.4263353 ,  0.45866269, ..., -0.37149517,
        -0.37149517, -0.37149517],
       [ 0.1902498 ,  0.30219991,  0.43098798, ..., -0.37149517,
        -0.37149517, -0.37149517],
       [ 0.08980027,  0.2163518 ,  0.36026655, ..., -0.37149517,
        -0.37149517, -0.37149517]])

In [137]:
max(sol_vect2.transpose()[0]-sol_vect.y[0])

np.float64(0.0011785738106997456)

In [123]:
sol_for_loop[0].y

array([[ 0.19260168, -0.37420252, -1.39851976, ..., -1.29440028,
        -1.29442285, -1.29441762],
       [ 0.73680123,  0.69675917,  0.59866276, ..., -0.37149902,
        -0.37150226, -0.37150455]])

In [130]:
max(sol_vect.y[-1]-sol_for_loop[-1].y[-1])

np.float64(0.0005143403928675183)

In [124]:
sol_vect.y[int(len(sol_vect.y)/2)]

array([ 0.73680123,  0.69677437,  0.59867734, ..., -0.37149517,
       -0.37149517, -0.37149517])